In [1]:
from bs4 import BeautifulSoup
import requests, string, math, os, shutil

In [2]:
URL = "https://skybrary.aero"
webSite = requests.get(URL)
soup = BeautifulSoup(webSite.content, "html.parser")

# find all a tags in div with class="sf-depth-1 menuparent" or "sf-depth-1"
navigations = soup.find_all("a", class_=["sf-depth-1 menuparent", "sf-depth-1"])

dict_navigations={}

for navigation in navigations:
    dict_navigations[navigation.text.strip()] = URL + navigation["href"]

dict_navigations

{'Portals': 'https://skybrary.aero/portals',
 'Browsers': 'https://skybrary.aero/browsers',
 'Themes': 'https://skybrary.aero/themes-0',
 'Tutorials': 'https://skybrary.aero/tutorials',
 'Courses': 'https://skybrary.aero/courses',
 'Publications': 'https://skybrary.aero/publications'}

In [3]:
portals_page = requests.get(dict_navigations["Portals"])
portals_soup = BeautifulSoup(portals_page.content, "html.parser")

# find all a tags in div with class="term-name"
portal_categories = portals_soup.find_all("div", class_="term-name")

array_portal_categories=[]
for portal_category in portal_categories:
    # add link to array_portal_categories
    array_portal_categories.append([URL + portal_category.find("a")["href"], portal_category.text.strip()])

array_portal_categories

[['https://skybrary.aero/operational-issues/air-ground-communication',
  'Air Ground Communication'],
 ['https://skybrary.aero/operational-issues/airspace-infringement',
  'Airspace Infringement'],
 ['https://skybrary.aero/operational-issues/wildlife-strike',
  'Wildlife Strike'],
 ['https://skybrary.aero/operational-issues/controlled-flight-terrain',
  'Controlled Flight Into Terrain'],
 ['https://skybrary.aero/operational-issues/fire-smoke-and-fumes',
  'Fire Smoke and Fumes'],
 ['https://skybrary.aero/operational-issues/ground-operations',
  'Ground Operations'],
 ['https://skybrary.aero/operational-issues/airworthiness', 'Airworthiness'],
 ['https://skybrary.aero/operational-issues/level-bust', 'Level Bust'],
 ['https://skybrary.aero/operational-issues/loss-control', 'Loss of Control'],
 ['https://skybrary.aero/operational-issues/loss-separation',
  'Loss of Separation'],
 ['https://skybrary.aero/operational-issues/runway-excursion',
  'Runway Excursion'],
 ['https://skybrary.aero/

In [23]:
def ParserCategory(URL, name_category):
    
    print("PARSING: "+URL)
    category_page = requests.get(URL+"?items_per_page=100")
    category_soup = BeautifulSoup(category_page.content, "html.parser")
    
    articles=[]

    if category_soup.find("li", class_="pager__item pager__item--last"):
        # find number of pages
        number_of_pages = int(category_soup.find("a", title="Go to last page")["href"].split("&page=")[1]) +1
        print("Nombre de pages: "+str(number_of_pages))

        for i in range(int(number_of_pages)):
            category_page = requests.get(URL+"?items_per_page=100&page="+str(i))
            category_soup = BeautifulSoup(category_page.content, "html.parser")
            div_articles = category_soup.find_all("div", class_="title")
            span_articles = category_soup.find_all("span", class_="field-content")
            articles = articles + div_articles + span_articles
    
    else:
        div_articles = category_soup.find_all("div", class_="title")
        span_articles = category_soup.find_all("span", class_="field-content")
        articles = div_articles + span_articles

    print("Nombre d'articles: "+str(len(articles)))
    #print(articles)
    
    for article in articles:
        
        URL_article = "https://skybrary.aero" + article.find("a")["href"]
        webSite_article = requests.get(URL_article)
        print("PARSING: "+URL_article)
        soup_article = BeautifulSoup(webSite_article.content, "html.parser")
        group_items = soup_article.find("div", class_="group-left-bottom")
        
        # si group_items est non vide
        if group_items:
            
            technical_info = group_items.find("div", class_="group-technical-data data-table")

            # Initialiser un dictionnaire pour stocker le contenu des <h2> et <p>
            data_dict = {}

            # Parcourir le contenu HTML pour récupérer le contenu des <h2> et <p>
            current_h2 = None
            current_p = []
            for tag in group_items.find_all(['h2', 'p', 'li', 'th', 'td', "div"]):
                if tag.name == 'h2':
                    # Enregistrer le contenu du dernier <h2> et ses <p> associés
                    if current_h2 is not None and current_p:
                        data_dict[current_h2] = current_p

                    # Mettre à jour la valeur du dernier <h2>
                    current_h2 = tag.text.strip()

                    # Réinitialiser la liste des <p> associés au nouveau <h2>
                    current_p = []
                
                elif tag.name == 'p':
                    # Ajouter le contenu du <p> à la liste en cours
                    current_p.append(tag.text.strip())

                elif tag.name == 'li':
                    # Ajouter le contenu du <li> à la liste en cours
                    current_p.append(tag.text.strip()+".")
                
                elif tag.name == 'th':
                    current_p.append(tag.text.strip()+":")
                
                elif tag.name == 'td':
                    current_p.append(tag.text.strip()+".")

                elif tag == technical_info:

                    labels = tag.find_all("div", class_="field-label")
                    values = tag.find_all("div", class_="field-item")
                    
                    for label, value in zip(labels, values):
                        current_p.append(label.text.strip()+": "+value.text.strip()+".")

            # Sauvegarder le dernier <h2> et ses <p> associés (s'il y en a)
            if current_h2 is not None and current_p:
                data_dict[current_h2] = current_p

            # Écrire les données dans des fichiers .txt
            for h2, p_content in data_dict.items():
                # Utiliser le nom du fichier basé sur la valeur de <h2> (sans caractères invalides)
                filename_prefix = f"{h2.translate(str.maketrans('', '', string.punctuation))}"

                # Limiter le contenu de <p> à 500 mots par fichier
                words_limit = 500
                total_words = len(" ".join(p_content).split())
                num_files = math.ceil(total_words / words_limit)

                for i in range(num_files):
                    # Sélectionner les mots pour chaque fichier
                    start_idx = i * words_limit
                    end_idx = (i + 1) * words_limit
                    words = " ".join(p_content).split()[start_idx:end_idx]

                    # Construire le contenu du fichier
                    file_content = " ".join(words)

                    # Numéroter le fichier s'il y a plus d'un fichier
                    if num_files > 1:
                        filename = f"/{filename_prefix}_{i+1}.txt"
                    else:
                        filename = f"/{filename_prefix}.txt"

                    # Si répertoire name_category n'existe pas alors le créer
                    if os.path.exists("Skybrary_data/"+name_category) == False:
                        os.mkdir("Skybrary_data/"+name_category)

                    # Si répertoire article.text n'existe pas alors le créer
                    directory_name = f"{article.text.translate(str.maketrans('', '', string.punctuation))}"
                    if os.path.exists("Skybrary_data/"+name_category+"/"+directory_name) == False:
                        os.mkdir("Skybrary_data/"+name_category+"/"+directory_name)

                    # Écrire le contenu dans le fichier .txt
                    with open("Skybrary_data/"+name_category +"/"+ directory_name + filename, 'w', encoding='utf-8') as file:
                        file.write(file_content)

In [5]:
if os.path.exists("Skybrary_data") == True:
    #supprimer le dossier data et son contenu
    shutil.rmtree("Skybrary_data")
# Créer le dossier data
os.mkdir("Skybrary_data")

In [24]:
for category in array_portal_categories:
    ParserCategory(category[0], category[1])

PARSING: https://skybrary.aero/operational-issues/air-ground-communication
Nombre d'articles: 71
PARSING: https://skybrary.aero/articles/accident-and-serious-incident-reports-agc
PARSING: https://skybrary.aero/articles/aeronautical-frequency-protection-interference
PARSING: https://skybrary.aero/articles/air-ground-communication
PARSING: https://skybrary.aero/articles/air-ground-voice-communications
PARSING: https://skybrary.aero/articles/aircraft-call-sign
PARSING: https://skybrary.aero/articles/aircraft-communications-addressing-and-reporting-system
PARSING: https://skybrary.aero/articles/atc-active-listening
PARSING: https://skybrary.aero/articles/atco-actions-during-prolonged-loss-communication
PARSING: https://skybrary.aero/articles/ats-unit-callsigns
PARSING: https://skybrary.aero/articles/automatic-dependent-surveillance-broadcast-ads-b
PARSING: https://skybrary.aero/articles/automatic-dependent-surveillance-contract-ads-c
PARSING: https://skybrary.aero/articles/automatic-termin

KeyboardInterrupt: 

In [7]:
browsers_page = requests.get(dict_navigations["Browsers"])
browsers_soup = BeautifulSoup(browsers_page.content, "html.parser")

# find all <a> tags in div with class="filed-item"
browsers = browsers_soup.find("div", class_="group-inner").find_all("a")
browsers

array_browsers=[]
for browser in browsers:
    array_browsers.append([URL + browser["href"], browser.text.strip()])

array_browsers

[['https://skybrary.aero/accidents-and-incidents', 'Accidents and Incidents'],
 ['https://skybrary.aero/aircraft-types', 'Aircraft types'],
 ['https://skybrary.aero/airports', 'Airports'],
 ['https://skybrary.aero/territories', 'ICAO Member States and Territories'],
 ['https://skybrary.aero/operators', 'Aircraft operators'],
 ['https://skybrary.aero/bookshelf-items', 'Bookshelf']]

In [25]:
# ACCIDENTS AND INCIDENTS
print(array_browsers[0])
ParserCategory(array_browsers[0][0], array_browsers[0][1])

['https://skybrary.aero/accidents-and-incidents', 'Accidents and Incidents']
PARSING: https://skybrary.aero/accidents-and-incidents
Nombre de pages: 15
Nombre d'articles: 1415
PARSING: https://skybrary.aero/accidents-and-incidents/a109-vicinity-london-heliport-london-uk-2013
PARSING: https://skybrary.aero/accidents-and-incidents/a124-zaragoza-spain-2010
PARSING: https://skybrary.aero/accidents-and-incidents/a139-a30b-ottawa-canada-2014
PARSING: https://skybrary.aero/accidents-and-incidents/a139-vicinity-sky-shuttle-heliport-hong-kong-china-2010
PARSING: https://skybrary.aero/accidents-and-incidents/a140-vicinity-tehran-mehrabad-iran-2014
PARSING: https://skybrary.aero/accidents-and-incidents/a169-leicester-uk-2018
PARSING: https://skybrary.aero/accidents-and-incidents/a20n-a320-amsterdam-netherlands-2019
PARSING: https://skybrary.aero/accidents-and-incidents/a20n-ac11-en-route-northwest-zurich-german-airspace-2019
PARSING: https://skybrary.aero/accidents-and-incidents/a20n-atlantic-cit

KeyboardInterrupt: 

In [27]:
# AIRCRAFT
print(array_browsers[1])
ParserCategory(array_browsers[1][0], array_browsers[1][1])

['https://skybrary.aero/aircraft-types', 'Aircraft types']
PARSING: https://skybrary.aero/aircraft-types
Nombre de pages: 6
Nombre d'articles: 576
PARSING: https://skybrary.aero/aircraft/f260
PARSING: https://skybrary.aero/aircraft/m339
PARSING: https://skybrary.aero/aircraft/ac56
PARSING: https://skybrary.aero/aircraft/l159
PARSING: https://skybrary.aero/aircraft/ac68
PARSING: https://skybrary.aero/aircraft/sgup
PARSING: https://skybrary.aero/aircraft/l39
PARSING: https://skybrary.aero/aircraft/alo2
PARSING: https://skybrary.aero/aircraft/alo3
PARSING: https://skybrary.aero/aircraft/as32
PARSING: https://skybrary.aero/aircraft/as3b
PARSING: https://skybrary.aero/aircraft/as50


KeyboardInterrupt: 

In [28]:
# AIRPORTS
print(array_browsers[2])
ParserCategory(array_browsers[2][0], array_browsers[2][1])

['https://skybrary.aero/airports', 'Airports']
PARSING: https://skybrary.aero/airports
Nombre de pages: 48
Nombre d'articles: 4746
PARSING: https://skybrary.aero/airports/leco
PARSING: https://skybrary.aero/airports/edka
PARSING: https://skybrary.aero/airports/ekyt
PARSING: https://skybrary.aero/airports/ekah
PARSING: https://skybrary.aero/airports/hajm
PARSING: https://skybrary.aero/airports/hadr
PARSING: https://skybrary.aero/airports/oiaa
PARSING: https://skybrary.aero/airports/lfoi
PARSING: https://skybrary.aero/airports/cyxx
PARSING: https://skybrary.aero/airports/htza
PARSING: https://skybrary.aero/airports/musc
PARSING: https://skybrary.aero/airports/diau
PARSING: https://skybrary.aero/airports/egpd
PARSING: https://skybrary.aero/airports/kabr
PARSING: https://skybrary.aero/airports/oeab
PARSING: https://skybrary.aero/airports/diap
PARSING: https://skybrary.aero/airports/kabi
PARSING: https://skybrary.aero/airports/diao
PARSING: https://skybrary.aero/airports/oebq
PARSING: https

In [ ]:
# ICAO member states
print(array_browsers[3])
ParserCategory(array_browsers[3][0], array_browsers[3][1])

In [ ]:
# AIRCRAFT OPERATORS
print(array_browsers[4])
ParserCategory(array_browsers[4][0], array_browsers[4][1])